In [1]:
import os
import rosbag

from sensor_msgs.msg import Image
import cv2
import numpy as np

Failed to load Python extension for LZ4 support. LZ4 compression will not be available.


In [2]:
def extract_rgb_depth_data(bag_file):
    bag = rosbag.Bag(bag_file, 'r')

    rgb_data = []
    depth_data = []

    for topic, msg, t in bag.read_messages(topics=['/device_0/sensor_0/Color_0/image/data', '/device_0/sensor_0/Depth_0/image/data']):
        if "Color_0" in topic:
            rgb_data.append((t, msg))
        elif "Depth_0" in topic:
            depth_data.append((t, msg))

    bag.close()

    return rgb_data, depth_data

In [3]:
def save_images(rgb_data, depth_data, path, max_files=3):
    
    count = 0
    for t, rgb_msg in rgb_data:
        if count == max_files:
            break
        rgb_image = np.frombuffer(rgb_msg.data, dtype=np.uint8).reshape((rgb_msg.height, rgb_msg.width, 3))
        print(rgb_image.shape)
        rgb_image = rgb_image[:,:640,:]
        print(rgb_image.shape)
        cv2.imwrite(os.path.join(path, f'images/rgb_{t}.png'), cv2.cvtColor(rgb_image, cv2.COLOR_RGB2BGR))
        count += 1
        
    count = 0
    for t, depth_msg in depth_data:
        if count == max_files:
            break
        depth_image = np.frombuffer(depth_msg.data, dtype=np.uint16).reshape((depth_msg.height, depth_msg.width))
        cv2.imwrite(os.path.join(path, f'depth/depth_{t}.png'), depth_image)
        count += 1


In [4]:
def get_rosbag_topics(bag_file):
    bag = rosbag.Bag(bag_file, 'r')
    
    topics = bag.get_type_and_topic_info().topics.keys()

    bag.close()

    return topics

In [11]:
root = "./data"
specific_bag = "aruco_container3.bag"
bag_path = os.path.join(root, specific_bag)

bag = rosbag.Bag(bag_path, 'r')

In [12]:
topics_list = ['/device_0/sensor_0/Color_0/info/camera_info']
for topic, msg, t in bag.read_messages(topics=topics_list):
    print(topic)
    print(msg)
    print(t)

/device_0/sensor_0/Color_0/info/camera_info
header: 
  seq: 0
  stamp: 
    secs: 0
    nsecs:         0
  frame_id: ''
height: 480
width: 848
distortion_model: "Inverse Brown Conrady"
D: [-0.053771115839481354, 0.05535884201526642, 0.0008205653284676373, 0.0009527259971946478, -0.018054140731692314]
K: [433.34295654296875, 0.0, 418.0463562011719, 0.0, 432.9460754394531, 244.36111450195312, 0.0, 0.0, 1.0]
R: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
P: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
binning_x: 0
binning_y: 0
roi: 
  x_offset: 0
  y_offset: 0
  height: 0
  width: 0
  do_rectify: False
1


In [13]:
rgb_data, depth_data = extract_rgb_depth_data(bag_path)
save_images(rgb_data, depth_data, './data')

(480, 848, 3)
(480, 640, 3)
(480, 848, 3)
(480, 640, 3)
(480, 848, 3)
(480, 640, 3)
